In [47]:
from HLsearch import *
from scipy.integrate import solve_ivp
import numpy as np
from sympy import symbols, var, diff, simplify, collect,solve
from sympy.utilities.lambdify import lambdify, implemented_function

from operator import add,sub,mul

import itertools
import scipy.io as sio

import time

from scipy.linalg import null_space

def generate_data(func, time, init_values):
    sol = solve_ivp(func,[time[0],time[-1]],init_values,t_eval=time, method='RK45',rtol=1e-10,atol=1e-10)
    return sol.y.T, np.array([func(0,sol.y.T[i,:]) for i in range(sol.y.T.shape[0])],dtype=np.float64)

g=9.81
mp=0.1
mc=1
l=1

def cartpole(t,y,f=0.0):
    theta,thetadot,x,xdot = y
    xdotdot = (f+mp*np.sin(theta)*(l*thetadot**2+g*np.cos(theta)))/(mc+mp*np.sin(theta)**2)
    thetadotdot = (-f*np.cos(theta)-mp*l*thetadot**2*np.cos(theta)*np.sin(theta)-(mc+mp)*g*np.sin(theta))/(l*(mc+mp*np.sin(theta)**2))
    return thetadot,thetadotdot,xdot,xdotdot

t = np.arange(0,0.5,0.01)
y0=np.array([np.pi/4, 0,0, 0])
X,Xdot = generate_data(cartpole,t,y0)

# g = 9.81
# m = 1
# L = 1

# def spherePend(t,y,Moment=0.0):
#     theta, theta_t, phi, phi_t = y
#     theta_2t,phi_2t = (-L**2*m*np.sin(theta)*np.cos(theta)*theta_t**2 + L*g*m*np.sin(theta) + L*m*(-L*np.sin(phi)*np.sin(theta)*phi_t + L*np.cos(phi)*np.cos(theta)*theta_t)*np.sin(phi)*np.cos(theta)*phi_t + L*m*(-L*np.sin(phi)*np.sin(theta)*phi_t + L*np.cos(phi)*np.cos(theta)*theta_t)*np.sin(theta)*np.cos(phi)*theta_t + L*m*(L*np.sin(phi)*np.cos(theta)*theta_t + L*np.sin(theta)*np.cos(phi)*phi_t)*np.sin(phi)*np.sin(theta)*theta_t - L*m*(L*np.sin(phi)*np.cos(theta)*theta_t + L*np.sin(theta)*np.cos(phi)*phi_t)*np.cos(phi)*np.cos(theta)*phi_t - L*m*(-L*np.sin(phi)*np.sin(theta)*phi_t**2 - L*np.sin(phi)*np.sin(theta)*theta_t**2 + 2*L*np.cos(phi)*np.cos(theta)*phi_t*theta_t)*np.sin(phi)*np.cos(theta) - L*m*(-2*L*np.sin(phi)*np.cos(theta)*phi_t*theta_t - L*np.sin(theta)*np.cos(phi)*phi_t**2 - L*np.sin(theta)*np.cos(phi)*theta_t**2)*np.cos(phi)*np.cos(theta) + m*(-L*np.sin(phi)*np.sin(theta)*phi_t + L*np.cos(phi)*np.cos(theta)*theta_t)*(-2*L*np.sin(phi)*np.cos(theta)*phi_t - 2*L*np.sin(theta)*np.cos(phi)*theta_t)/2 + m*(-2*L*np.sin(phi)*np.sin(theta)*theta_t + 2*L*np.cos(phi)*np.cos(theta)*phi_t)*(L*np.sin(phi)*np.cos(theta)*theta_t + L*np.sin(theta)*np.cos(phi)*phi_t)/2)/(L**2*m*np.sin(phi)**2*np.cos(theta)**2 + L**2*m*np.sin(theta)**2 + L**2*m*np.cos(phi)**2*np.cos(theta)**2),(L*m*(-L*np.sin(phi)*np.sin(theta)*phi_t + L*np.cos(phi)*np.cos(theta)*theta_t)*np.sin(phi)*np.cos(theta)*theta_t + L*m*(-L*np.sin(phi)*np.sin(theta)*phi_t + L*np.cos(phi)*np.cos(theta)*theta_t)*np.sin(theta)*np.cos(phi)*phi_t + L*m*(L*np.sin(phi)*np.cos(theta)*theta_t + L*np.sin(theta)*np.cos(phi)*phi_t)*np.sin(phi)*np.sin(theta)*phi_t - L*m*(L*np.sin(phi)*np.cos(theta)*theta_t + L*np.sin(theta)*np.cos(phi)*phi_t)*np.cos(phi)*np.cos(theta)*theta_t - L*m*(-L*np.sin(phi)*np.sin(theta)*phi_t**2 - L*np.sin(phi)*np.sin(theta)*theta_t**2 + 2*L*np.cos(phi)*np.cos(theta)*phi_t*theta_t)*np.sin(theta)*np.cos(phi) + L*m*(-2*L*np.sin(phi)*np.cos(theta)*phi_t*theta_t - L*np.sin(theta)*np.cos(phi)*phi_t**2 - L*np.sin(theta)*np.cos(phi)*theta_t**2)*np.sin(phi)*np.sin(theta) + Moment + m*(-2*L*np.sin(phi)*np.sin(theta)*phi_t + 2*L*np.cos(phi)*np.cos(theta)*theta_t)*(L*np.sin(phi)*np.cos(theta)*theta_t + L*np.sin(theta)*np.cos(phi)*phi_t)/2 + m*(-L*np.sin(phi)*np.sin(theta)*phi_t + L*np.cos(phi)*np.cos(theta)*theta_t)*(-2*L*np.sin(phi)*np.cos(theta)*theta_t - 2*L*np.sin(theta)*np.cos(phi)*phi_t)/2)/(L**2*m*np.sin(phi)**2*np.sin(theta)**2 + L**2*m*np.sin(theta)**2*np.cos(phi)**2)
#     return theta_t,theta_2t,phi_t,phi_2t

# t = np.arange(0,5,0.01)
# y0=np.array([np.pi/4, 0,0, 0.5])
# X,Xdot = generate_data(spherePend,t,y0)

data_description = ()
for i in range(round(X.shape[1]/2)):
    data_description = data_description + symbols('x{}, x{}_t'.format(i,i))
print('Variables are:',data_description)
data_description_sym = data_description
data_description = list(str(descr) for descr in data_description)
# print(data_description_sym)
# print([data_description[i] for i in range(round(len(data_description)/2))])

expr_new0 = buildFunctionExpressions(1,round(X.shape[1]/2),[data_description[i] for i in range(round(len(data_description)/2))],use_sine=True)
expr_new1 = buildFunctionExpressions(1,round(X.shape[1]/2),[data_description[i] for i in range(round(len(data_description)/2),len(data_description))],use_sine=False)

print(expr_new0[1:])
print(expr_new1)
expr_new = expr_new0[1:]+expr_new1
expr_new = buildFunctionExpressions(3,len(expr_new),expr_new)


# expr_new0 = buildFunctionExpressions(1,round(X.shape[1]/2),[data_description[i] for i in range(round(len(data_description)/2))],use_sine=True)
# expr_new1 = buildFunctionExpressions(1,round(X.shape[1]/2),[data_description[i] for i in range(round(len(data_description)/2),len(data_description))],use_sine=False)

# print(expr_new0[1:])
# print(expr_new1)
# expr_new = expr_new0[1:]+expr_new1
# expr_new = buildFunctionExpressions(4,len(expr_new),expr_new)

Theta = buildTimeSerieMatrixFromFunctions(X,expr_new, data_description)
Gamma= buildTimeDerivativeMatrixFromFunctions(X,Xdot,expr_new,data_description)

# u, s, vh = np.linalg.svd(0*Theta-Gamma,full_matrices=False)

# xi = vh.T[:,-1]
# print(s)
# print(vh.T)

Variables are: (x0, x0_t, x1, x1_t)
['x0_t', 'sin(x0)', 'cos(x0)']
['x1', 'x1_t']


In [48]:
nullspace = null_space(Gamma)

In [88]:
print(nullspace.shape)

(55, 29)


In [86]:
def adm(subspace, qinit, lmbda, max_iter, tol):
        """
        :param subspace: a subspace for finding sparsest vector
        :param qinit: initialize coefficient matrix
        :param lmbda: threshold for doing adm
        :param max_iter: max iteration number for adm
        :param tol: tolerance for stopping criteria for adm
        :return: find a sparse representation
        """
        _q = qinit
        for _ in range(max_iter):
            q_old = _q
            soft_thresholding = lambda mat, lower: \
                np.multiply(np.sign(mat), np.maximum(np.absolute(mat) - lower, 0))
            _xi = soft_thresholding(np.matmul(subspace, _q), lmbda)
            _q = np.matmul(subspace.T, _xi)/(np.linalg.norm(np.matmul(subspace.T, _xi), ord=2))
            res_q = np.linalg.norm(q_old - _q, ord=2)
            if res_q <= tol:
                return _q
        print('warning, cant find optimum q within number of max iteration')
        return _q

def adm_initvary(null_space, lmbda, max_iter, tol):
    """
    :param self:
    :param null_space:
    :param lmbda: threshold for doing adm
    :param max_iter: max iteration number for adm
    :param tol: tolerance for stopping criteria for adm
    :return: find a sparse representation
    """
    ntn = np.empty(null_space.shape)
    _q = np.empty((null_space.shape[1], null_space.shape[0]))
    out = np.empty((null_space.shape[0], null_space.shape[0]))
    nzeros = np.empty((1, null_space.shape[0]))
    for _ii in range(null_space.shape[1]):
        ntn[:, _ii] = null_space[:, _ii]/np.mean(null_space[:, _ii])

    for _jj in range(ntn.shape[0]):
        # initial conditions
        qinit = ntn[_jj, :].T
        # algorithm for finding coefficients resulting in  sparsest vector in null space
        _q[:, _jj] = adm(null_space, qinit, lmbda, max_iter, tol)
        # compose sparest vectors
        out[:, _jj] = np.matmul(null_space, _q[:, _jj])
        # check how many zeros each of the found sparse vectors have
        nzeros[0, _jj] = np.count_nonzero(abs(out[:, _jj]) < lmbda)

    # find the vector with the largest number of zeros
    indsparse = np.nonzero((nzeros == np.max(nzeros)).flatten())[0]

    # save the indices of non zero coefficients
    indtheta = np.nonzero((np.absolute(out[:, indsparse[0]]) >= lmbda))[0]
    # get sparsest vector
    _xi = out[:, indsparse]
    smallinds = (abs(out[:, indsparse]) < lmbda)
    # set threshold coefficients to zero
    _xi[smallinds] = 0

    # check that the solution found by ADM is unique.
    if len(indsparse) > 1:
        xidiff = (out[indtheta, indsparse[0]] - out[indtheta, indsparse[1]])
        if np.all(xidiff > tol) == 1:
            print('ADM has discovered ',len(indsparse),' sparsest vectors')

    # calculate how many terms are in the sparsest vector
#        numterms = len(indtheta)
    return _xi

coef = adm_initvary(nullspace, 0.00087, 1000, 2e-3)

In [87]:
print(coef)
for i in range(coef.shape[1]):
    print(generateExpression(coef[:,i],expr_new,threshold=1e-12))
# print(generateExpression(coef[:,1],expr_new,threshold=1e-12))

[[ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 1.46276727e-02]
 [ 0.00000000e+00]
 [ 1.21321310e-03]
 [-1.11125476e-02]
 [ 0.00000000e+00]
 [ 8.70602424e-03]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [-5.54207185e-02]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 6.98561752e-03]
 [-5.91958216e-02]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 1.18929206e-01]
 [ 0.00000000e+00]
 [-1.46037493e-03]
 [ 8.13303574e-03]
 [ 9.60397444e-04]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [-4.56189474e-03]
 [-3.34642084e-03]
 [ 0.00000000e+00]
 [-1.57455762e-02]
 [ 9.48364292e-02]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [-4.27290072e-02]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [-2.09463452e-03]
 [ 0.00000000e+00]
 [ 9.07416122e-02]
 [ 1.18632999e-02]
 [-1.14572186e-03]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 9.79230716e-01]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 1.59372732e-03]
 [ 0.00000000e+00]
 [ 0.0000000